In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash_slicer import VolumeSlicer
from dash.dependencies import Input, Output, State, ALL
import imageio
import dash_daq as daq

app = JupyterDash(__name__, update_title=None)
server = app.server

# Read volume
vol = imageio.volread("imageio:stent.npz")
vol = vol[::3, :, :]
#spacing = 3, 1, 1
spacing = 1, 1, 1
#ori = 1000, 2000, 3000
ori = 0, 0, 0

# Create slicer objects
slicer0 = VolumeSlicer(app, vol, spacing=spacing, origin=ori, axis=0, thumbnail=False)
slicer1 = VolumeSlicer(
    app, vol, spacing=spacing, origin=ori, axis=1, thumbnail=8, reverse_y=False
)
slicer2 = VolumeSlicer(app, vol, spacing=spacing, origin=ori, axis=2, color="#00ff99")

setpos_store = dcc.Store(
    id={"context": "app", "scene": slicer0.scene_id, "name": "setpos"}
)

app.layout = html.Div(style={
    "display": "grid",
    "gridTemplateColumns": "33% 33% 33%",
}, children=[
    html.Div(
        [
            slicer0.graph,
            html.Div(slicer0.slider, style={"display": "none"}),
            *slicer0.stores
        ]
    ),
    html.Div(
        [
            slicer1.graph,
            html.Div(slicer1.slider, style={"display": "none"}),
            *slicer1.stores
        ]
    ),
    html.Div(
        [
            slicer2.graph,
            html.Div(slicer2.slider, style={"display": "none"}),
            *slicer2.stores
        ]
    ),
    setpos_store,
    html.Div([
        html.Pre("Voxel Coordinates (x, y, z):"),
        daq.NumericInput(
            id={
                'type': 'voxel_value',
                'index': 0
            },
            min=0,
            max=100,
            value=5
        ),
        daq.NumericInput(
            id={
                'type': 'voxel_value',
                'index': 1
            },
            min=0,
            max=100,
            value=5
        ),
        daq.NumericInput(
            id={
                'type': 'voxel_value',
                'index': 2
            },
            min=0,
            max=100,
            value=5,
        ),
    ]),
    html.Pre(id="write_slicer"),
    html.Pre(id="write_numericinput"),
]
)


# Jump slicers to position written in the numeric inputs
@app.callback(
    [
        Output("write_numericinput", "children"),
        Output(setpos_store.id, "data"),
    ],
    Input({"type": "voxel_value", "index": ALL}, "value"),
    prevent_initial_call=False
)
def write_slicer_pos(pos_values):
    x_pos, y_pos, z_pos = pos_values
    return f"Numeric Input values\nx: {x_pos}\ny: {y_pos}\nz: {z_pos}", (x_pos, y_pos, z_pos)



# Show the current slicer coordinates in the numeric input fields
@app.callback(
    Output({"type": "voxel_value", "index": ALL}, "value"),
    [
        Input(slicer0.graph.id, "clickData"),
        Input(slicer1.graph.id, "clickData"),
        Input(slicer2.graph.id, "clickData"),
    ],
    [
        State({"scene": slicer0.scene_id, "context": ALL, "name": "state"}, "data"),
    ],
    prevent_initial_call=True
)
def read_slicer_pos(fire1, fire2, fire3, slicer_index):
    z_pos = slicer_index[0]["index"]
    y_pos = slicer_index[1]["index"]
    x_pos = slicer_index[2]["index"]

    #z_world, y_world, x_world = [val * mult + orig for mult, orig, val in zip(spacing, ori, [z_pos, y_pos, x_pos])]
    return x_pos, y_pos, z_pos


@app.callback(
    Output("write_slicer", "children"),
    Input({"scene": slicer0.scene_id, "context": ALL, "name": "state"}, "data"),
)
def write_coords(slicer_index):
    if slicer_index is None or any([i is None for i in slicer_index]):
        return dash.no_update
    z_pos = slicer_index[0]["index"]
    y_pos = slicer_index[1]["index"]
    x_pos = slicer_index[2]["index"]
    return f"Direct output from slicers\nx: {x_pos}\ny: {y_pos}\nz: {z_pos}"


app.run_server(debug=True, dev_tools_props_check=False, mode="inline")
